Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [52]:
class Model:
    def __init__(self, batch_size=128, reg_weight=1e-5):
        self.batch_size = batch_size
        self.num_hidden = num_hidden
        self.reg_weight = reg_weight
        self.graph = tf.Graph()        
        
        with self.graph.as_default():
            # Input data. For the training data, we use a placeholder that will be fed
            # at run time with a training minibatch.
            self.tf_train_dataset = tf.placeholder(tf.float32,
                                            shape=(self.batch_size, image_size * image_size))
            self.tf_train_labels = tf.placeholder(tf.float32, shape=(self.batch_size, num_labels))
            self.tf_valid_dataset = tf.constant(valid_dataset)
            self.tf_test_dataset = tf.constant(test_dataset)

        
    def make_logistic(self):
        with self.graph.as_default():
            # Variables.
            weights = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_labels]))
            biases = tf.Variable(tf.zeros([num_labels]))

            # Training computation.
            logits = tf.matmul(self.tf_train_dataset, weights) + biases


            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(labels=self.tf_train_labels, logits=logits))

            regularization = 0.5 * tf.reduce_sum(weights ** 2)
            self.loss += self.reg_weight * regularization

            # Optimizer.
            self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)

            # Predictions for the training, validation, and test data.
            self.train_prediction = tf.nn.softmax(logits)
            self.valid_prediction = tf.nn.softmax(
            tf.matmul(self.tf_valid_dataset, weights) + biases)
            self.test_prediction = tf.nn.softmax(tf.matmul(self.tf_test_dataset, weights) + biases)

    def make_neural_network(self):
        with self.graph.as_default():
            # Variables.
            weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_hidden]))
            biases_1 = tf.Variable(tf.zeros([num_hidden]))

            weights_2 = tf.Variable(
            tf.truncated_normal([num_hidden, num_labels]))
            biases_2 = tf.Variable(tf.zeros([num_labels]))

            # Training computation.
            hidden = tf.nn.relu(tf.matmul(self.tf_train_dataset, weights_1) + biases_1)
            logits = tf.matmul(hidden, weights_2) + biases_2
            self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=self.tf_train_labels, logits=logits))

            regularization = 0.5 * (tf.reduce_sum(weights_1 ** 2) + tf.reduce_sum(weights_2 ** 2))
            self.loss += self.reg_weight * regularization

            # Optimizer.
            self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)

            def predict(input_):
                return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(input_, weights_1) + biases_1), weights_2) + biases_2)

            # Predictions for the training, validation, and test data.
            self.train_prediction = tf.nn.softmax(logits)
            self.valid_prediction = predict(self.tf_valid_dataset)
            self.test_prediction = predict(self.tf_test_dataset)

    def run(self, num_steps):
        with tf.Session(graph=self.graph) as session:
          tf.global_variables_initializer().run()
          print("Initialized")
          for step in range(num_steps):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * self.batch_size) % (train_labels.shape[0] - batch_size)
            # Generate a minibatch.
            batch_data = train_dataset[offset:(offset + batch_size), :]
            batch_labels = train_labels[offset:(offset + batch_size), :]

            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {self.tf_train_dataset : batch_data, self.tf_train_labels : batch_labels}
            _, l, predictions = session.run(
              [self.optimizer, self.loss, self.train_prediction], feed_dict=feed_dict)
            if (step % 500 == 0):
              print("Minibatch loss at step %d: %f" % (step, l))
              print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
              print("Validation accuracy: %.1f%%" % accuracy(
                self.valid_prediction.eval(), valid_labels))
          print("Test accuracy: %.1f%%" % accuracy(self.test_prediction.eval(), test_labels))

batch_size = 128
num_steps = 3001

g = Model(batch_size)
g.make_neural_network()
g.run(num_steps)

g2 = Model(batch_size)
g2.make_logistic()
g2.run(num_steps)



Initialized
Minibatch loss at step 0: 320.863373
Minibatch accuracy: 10.2%
Validation accuracy: 39.0%


KeyboardInterrupt: 

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [37]:
123

123

In [43]:
class ModelOverfit(Model):
    def __init__(self, batch_size=128, reg_weight=0):
        Model.__init__(self, batch_size=batch_size, reg_weight=reg_weight)
        
    def run(self, num_steps):
        with tf.Session(graph=self.graph) as session:
            tf.global_variables_initializer().run()
            print("Initialized")
            _train_dataset = train_dataset[:500]
            _train_labels = train_labels[:500]
            for step in range(num_steps):
                # Pick an offset within the training data, which has been randomized.
                # Note: we could use better randomization across epochs.
                offset = (step * self.batch_size) % (_train_labels.shape[0] - batch_size)
                # Generate a minibatch.
                batch_data = _train_dataset[offset:(offset + batch_size), :]
                batch_labels = _train_labels[offset:(offset + batch_size), :]

                # Prepare a dictionary telling the session where to feed the minibatch.
                # The key of the dictionary is the placeholder node of the graph to be fed,
                # and the value is the numpy array to feed to it.
                feed_dict = {self.tf_train_dataset : batch_data, self.tf_train_labels : batch_labels}
                _, l, predictions = session.run(
                  [self.optimizer, self.loss, self.train_prediction], feed_dict=feed_dict)
                if (step % 500 == 0):
                    print("Minibatch loss at step %d: %f" % (step, l))
                    print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                    print("Validation accuracy: %.1f%%" % accuracy(
                      self.valid_prediction.eval(), valid_labels))
            print("Test accuracy: %.1f%%" % accuracy(self.test_prediction.eval(), test_labels))

In [46]:
batch_size = 128
num_steps = 1001

g = ModelOverfit(batch_size, reg_weight=1e-2)
g.make_neural_network()
g.run(num_steps)

g = ModelOverfit(batch_size, reg_weight=0)
g.make_neural_network()
g.run(num_steps)

Initialized
Minibatch loss at step 0: 3476.956787
Minibatch accuracy: 8.6%
Validation accuracy: 34.8%
Minibatch loss at step 500: 21.067148
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 0.488302
Minibatch accuracy: 100.0%
Validation accuracy: 78.3%
Test accuracy: 84.8%
Initialized
Minibatch loss at step 0: 379.484436
Minibatch accuracy: 10.2%
Validation accuracy: 29.3%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.9%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
Test accuracy: 80.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [56]:
class DropoutNetwork(Model, ModelOverfit):
    def __init__(self, batch_size, reg_weight):
        Model.__init__(self, batch_size, reg_weight)
        
    def make_neural_network(self):
        with self.graph.as_default():
            # Variables.
            weights_1 = tf.Variable(
            tf.truncated_normal([image_size * image_size, num_hidden]))
            biases_1 = tf.Variable(tf.zeros([num_hidden]))

            weights_2 = tf.Variable(
            tf.truncated_normal([num_hidden, num_labels]))
            biases_2 = tf.Variable(tf.zeros([num_labels]))

            # Training computation.
            hidden = tf.nn.relu(tf.matmul(self.tf_train_dataset, weights_1) + biases_1)
            dropped_hidden = tf.nn.dropout(hidden, keep_prob=0.5)
            logits = tf.matmul(dropped_hidden, weights_2) + biases_2
            self.loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=self.tf_train_labels, logits=logits))

            regularization = 0.5 * (tf.reduce_sum(weights_1 ** 2) + tf.reduce_sum(weights_2 ** 2))
            self.loss += self.reg_weight * regularization

            # Optimizer.
            self.optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(self.loss)

            def predict(input_):
                return tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(input_, weights_1) + biases_1), weights_2) + biases_2)

            # Predictions for the training, validation, and test data.
            self.train_prediction = tf.nn.softmax(logits)
            self.valid_prediction = predict(self.tf_valid_dataset)
            self.test_prediction = predict(self.tf_test_dataset)    

In [57]:
g = DropoutNetwork(batch_size=128, reg_weight=1e-3)
g.make_neural_network()
g.run(num_steps)

Initialized
Minibatch loss at step 0: 786.579529
Minibatch accuracy: 7.8%
Validation accuracy: 37.0%
Minibatch loss at step 500: 205.120178
Minibatch accuracy: 70.3%
Validation accuracy: 80.7%
Minibatch loss at step 1000: 115.284149
Minibatch accuracy: 78.1%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 70.731598
Minibatch accuracy: 74.2%
Validation accuracy: 80.5%
Minibatch loss at step 2000: 42.289986
Minibatch accuracy: 84.4%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 25.371387
Minibatch accuracy: 85.9%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 15.599737
Minibatch accuracy: 81.2%
Validation accuracy: 85.6%
Test accuracy: 91.6%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
